# 🔭 Spectral Fitting Analysis (Google Colab)

Notebook สำหรับวิเคราะห์และ fit สเปกตรัมรังสีเอกซ์ของ AGN
รองรับการกำหนดช่วง Energy Range และเลือก Model ได้เอง

### 📝 วิธีใช้:
1. **Upload** โฟลเดอร์ `Colab_Spectral_Fitting` ทั้งหมดไปที่ Google Drive
2. **Mount Drive** (รัน Cell แรก)
3. **กำหนดค่า** Energy Range และ File Paths ใน Section 1
4. **Load Data** ใน Section 2 (อ่านข้อมูลเข้ามาก่อน)
5. **ทดลอง Model** ใน Section 3 (ทีละโมเดล)
6. **Final Fit** ใน Section 4 (รันสรุปผล)

---

## 0. Setup & Mount Drive

In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Change directory to where you uploaded the folder
import os
try:
    os.chdir('/content/drive/MyDrive/Colab_Spectral_Fitting')
    print("✅ Converted Correctly")
except:
    print("⚠️ Warning: Check your path /content/drive/MyDrive/...")

# Verify files
!ls -F

## 1. Configuration - กำหนดค่าก่อนรัน

In [ ]:
# ==============================
# 🎯 ENERGY RANGE (keV)
# กำหนดช่วงพลังงานที่จะวิเคราะห์
# ==============================
ENERGY_MIN = 0.3    # keV (ขอบล่าง)
ENERGY_MAX = 10.0   # keV (ขอบบน)

# ==============================
# 📁 FILE PATHS
# กำหนด path ไปยังไฟล์ข้อมูล
# ==============================
SOURCE_SPECTRUM_PATH = '/content/drive/MyDrive/data/source.pha'    # 📊 Source Spectrum
BACKGROUND_PATH = '/content/drive/MyDrive/data/background.pha'     # 🌌 Background Spectrum
ARF_PATH = '/content/drive/MyDrive/data/source.arf'                # 📈 ARF File
RMF_PATH = '/content/drive/MyDrive/data/source.rmf'                # 🔲 RMF File

# ==============================
# 🔧 FITTING PARAMETERS
# ==============================
N_STEPS = 10           # จำนวน steps สำหรับ grid search (5-20)
USE_BRUTE_FORCE = False # แนะนำ False (Optimization) สำหรับการลองเล่น

### 1.1 🔍 Check File Header (Optional)

In [ ]:
FILE_TO_INSPECT = SOURCE_SPECTRUM_PATH

from astropy.io import fits
import os

if os.path.exists(FILE_TO_INSPECT):
    print(f"📄 Inspecting: {os.path.basename(FILE_TO_INSPECT)}")
    print("=" * 60)
    try:
        with fits.open(FILE_TO_INSPECT) as hdul:
            hdul.info()
            ext_idx = 1 if len(hdul) > 1 else 0
            header = hdul[ext_idx].header
            print(f"\n📋 Key Headers:")
            for key in ['TELESCOP', 'INSTRUME', 'EXPOSURE', 'BACKSCAL']:
                if key in header: print(f"{key:<10} = {header[key]}")
    except Exception as e:
        print(f"❌ Error: {e}")
else:
    print("❌ File not found.")

## 2. Load & Prepare Data (WITH INSPECTION)
โหลดข้อมูล และแสดงกราฟเพื่อดูว่า Energy Range ที่เลือกครอบคลุมข้อมูลที่ดีหรือไม่

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import minimize
import sys
import importlib
import copy

# Import & Reload
import data_loader
import Models
importlib.reload(data_loader)
importlib.reload(Models)

from data_loader import (
    read_spectrum_file, 
    read_arf_file, 
    read_rmf_file,
    subtract_background,
    get_energy_from_response,
    filter_energy_range,
    fold_model_through_response
)
from Models import combined_model

print("📊 Loading Data...")

# 1. Load Files
source_spec = read_spectrum_file(SOURCE_SPECTRUM_PATH)
bkg_spec = read_spectrum_file(BACKGROUND_PATH)
arf_data = read_arf_file(ARF_PATH)
rmf_data = read_rmf_file(RMF_PATH)

# 2. Prepare Data
energy = get_energy_from_response(arf_data, rmf_data)

if source_spec and bkg_spec:
    net_counts, net_error = subtract_background(source_spec, bkg_spec)
else:
    net_counts = source_spec.counts
    net_error = np.sqrt(np.maximum(net_counts, 1.0))

exposure = source_spec.exposure if source_spec else 1.0
observed_rate = net_counts / exposure
observed_error = net_error / exposure

# -----------------------------------------------------
# 📈 PLOT DATA INSPECTION (NEW)
# เช็คกราฟตรงนี้เพื่อดูว่า Energy Range เหมาะสมไหม
# -----------------------------------------------------
# Align for plotting entire range
plot_len = min(len(energy), len(observed_rate))
plt.figure(figsize=(10, 4))
plt.errorbar(energy[:plot_len], observed_rate[:plot_len], yerr=observed_error[:plot_len], 
             fmt='.', color='gray', alpha=0.3, label='Full Data (Noisy)')

# Mark Selected Range
plt.axvline(ENERGY_MIN, color='blue', linestyle='--', lw=2, label=f'Min: {ENERGY_MIN} keV')
plt.axvline(ENERGY_MAX, color='blue', linestyle='--', lw=2, label=f'Max: {ENERGY_MAX} keV')

# Highlight Selected Data
mask_plot = (energy[:plot_len] >= ENERGY_MIN) & (energy[:plot_len] <= ENERGY_MAX)
plt.errorbar(energy[:plot_len][mask_plot], observed_rate[:plot_len][mask_plot], 
             yerr=observed_error[:plot_len][mask_plot], 
             fmt='o', color='black', markersize=2, label='Selected Data')

plt.xscale('log'); plt.yscale('log')
plt.xlabel('Energy (keV)'); plt.ylabel('Count Rate')
plt.title('🔍 Data Inspection: Check if selected range (Blue Dashed) covers good data')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()
# -----------------------------------------------------

# Filter Energy Range (Actual Processing)
energy_filtered, rate_filtered, error_filtered = filter_energy_range(
    energy, observed_rate, observed_error, 
    e_min=ENERGY_MIN, e_max=ENERGY_MAX
)

# Filter ARF
arf_filtered = None
if arf_data is not None:
    arf_filtered = copy.deepcopy(arf_data)
    min_len = min(len(energy), len(observed_rate))
    energy_aligned = energy[:min_len]
    mask = (energy_aligned >= ENERGY_MIN) & (energy_aligned <= ENERGY_MAX)
    
    if len(arf_filtered.arf) > len(mask):
         arf_filtered.arf = arf_filtered.arf[:len(mask)]
         if arf_filtered.energy_lo is not None: arf_filtered.energy_lo = arf_filtered.energy_lo[:len(mask)]
         if arf_filtered.energy_hi is not None: arf_filtered.energy_hi = arf_filtered.energy_hi[:len(mask)]
         if arf_filtered.energy_mid is not None: arf_filtered.energy_mid = arf_filtered.energy_mid[:len(mask)]

    arf_filtered.arf = arf_filtered.arf[mask]
    if arf_filtered.energy_mid is not None: arf_filtered.energy_mid = arf_filtered.energy_mid[mask]
    
print(f"✅ Data Ready: {len(energy_filtered)} bins selected")

## 3. 🧪 Step-by-Step Model Exploration
ใช้ฟังก์ชัน `fit_and_plot` เพื่อทดลองใส่แต่ละ Model ลงไปทีละขั้น

In [ ]:
# ========================================================
# 🧩 Core Functions: แยกส่วนคำนวณและอัลกอริทึมให้ชัดเจน
# ========================================================

def calculate_model_flux(params, energy, model_list, arf=None):
    """ 1. Model Calculation: คำนวณค่า Flux จากแบบจำลองทางฟิสิกส์ """
    # คำนวณ Flux ทฤษฎี
    flux = combined_model(energy, params, model_list)
    
    # นำไปคูณกับ ARF เพื่อจำลองสิ่งที่กล้องเห็น (Folding)
    if arf:
        predicted_rate = fold_model_through_response(flux, arf)
    else:
        predicted_rate = flux
    return predicted_rate

def calculate_chi_squared(observed, predicted, error):
    """ 2. Statistic: คำนวณค่าความผิดพลาด (Chi-Squared) """
    valid_mask = error > 0
    # สูตร Chi^2 = Sum((Data - Model)^2 / Error^2)
    chi2 = np.sum(((observed[valid_mask] - predicted[valid_mask]) / error[valid_mask])**2)
    return chi2

def find_best_parameters(initial_params, active_keys, model_list, bounds):
    """ 3. Optimization Algorithm: หาค่า Parameter ที่ทำให้ Chi-Squared ต่ำที่สุด """
    
    # ฟังก์ชันเป้าหมายที่จะ Optimize (Cost Function)
    def objective_function(param_values):
        # อัปเดตค่าปัจจุบันลง Dictionary
        current_p = initial_params.copy()
        for k, v in zip(active_keys, param_values):
            current_p[k] = v
            
        # 1. คำนวณ Model
        predicted = calculate_model_flux(current_p, energy_filtered, model_list, arf_filtered)
        
        # 2. คำนวณ Chi-Square
        return calculate_chi_squared(rate_filtered, predicted, error_filtered)

    # เริ่มรัน L-BFGS-B Algorithm
    start_values = [initial_params[k] for k in active_keys]
    result = minimize(objective_function, start_values, args=(), bounds=bounds, method='L-BFGS-B')
    
    return result

# ========================================================
# Main Helper: รวมทุกขั้นตอนเข้าด้วยกัน
# ========================================================
def fit_and_plot(model_list, prev_params=None, title="Model Test"):
    print(f"\n🧪 Testing Model: {model_list}")
    
    # 1. Setup Defaults
    current_params = {
        'pl_norm': 1e-3, 'photon_index': 1.7,
        'nH': 0.1,
        'bb_norm': 1e-4, 'kT': 0.1,
        'line_norm': 1e-4, 'line_energy': 6.4, 'line_sigma': 0.1,
        'refl_norm': 1e-4
    }
    if prev_params: current_params.update(prev_params)

    # 2. Identify Active Parameters
    active_keys = []
    if 'powerlaw' in model_list: active_keys.extend(['pl_norm', 'photon_index'])
    if 'tbabs' in model_list: active_keys.extend(['nH'])
    if 'blackbody' in model_list: active_keys.extend(['bb_norm', 'kT'])
    if 'gaussian' in model_list: active_keys.extend(['line_norm', 'line_energy'])
    if 'reflection' in model_list: active_keys.extend(['refl_norm'])
    
    # 3. Set Bounds (Physically Realistic)
    bound_map = {
        'photon_index': (1.0, 4.0), 
        'line_energy': (5.0, 7.5),
        'line_sigma': (0.01, 0.5),
        'kT': (0.01, 3.0),
        'nH': (0.0, 50.0),
    }
    bounds = [bound_map.get(k, (1e-8, 100.0)) for k in active_keys]

    # 4. Run Optimization
    c_res = find_best_parameters(current_params, active_keys, model_list, bounds)
    
    # Update Results
    best_p = current_params.copy()
    for k, v in zip(active_keys, c_res.x):
        best_p[k] = v
        
    red_chi2 = c_res.fun/(len(rate_filtered)-len(active_keys))
    print(f"✅ Reduced Chi-sq: {red_chi2:.2f}")
    
    # 5. Plot Results
    best_rate = calculate_model_flux(best_p, energy_filtered, model_list, arf_filtered)
    
    plt.figure(figsize=(8, 5))
    plt.errorbar(energy_filtered, rate_filtered, yerr=error_filtered, fmt='.', color='gray', alpha=0.5, label='Data')
    plt.plot(energy_filtered, best_rate, 'r-', lw=2, label=f'Model: {title}')
    plt.yscale('log'); plt.xscale('log')
    plt.xlabel('Energy (keV)'); plt.ylabel('Count Rate')
    plt.legend(); plt.title(f"{title} (Red $\chi^2$={red_chi2:.2f})")
    plt.grid(True, alpha=0.3)
    plt.show()
    
    return best_p

### 1. Powerlaw (Continuum)

In [ ]:
params_1 = fit_and_plot(['powerlaw'], title="1. Powerlaw")

### 2. + Reflection

In [ ]:
params_2 = fit_and_plot(['powerlaw', 'reflection'], prev_params=params_1, title="2. + Reflection")

### 3. + TBabs (Absorption)

In [ ]:
params_3 = fit_and_plot(['powerlaw', 'reflection', 'tbabs'], prev_params=params_2, title="3. + TBabs")

### 4. + Gaussian Line

In [ ]:
params_4 = fit_and_plot(['powerlaw', 'reflection', 'tbabs', 'gaussian'], prev_params=params_3, title="4. + Gaussian")

### 5. + Blackbody (Soft Excess)

In [ ]:
params_5 = fit_and_plot(['powerlaw', 'reflection', 'tbabs', 'gaussian', 'blackbody'], prev_params=params_4, title="5. + Blackbody")

## 4. Final Summary Fit

In [ ]:
print("✅ Final Optimized Parameters:")
for k, v in params_5.items():
    print(f"  {k:15s} : {v:.4f}")

# คำนวณ Fit Quality
final_model_flux = calculate_model_flux(params_5, energy_filtered, ['powerlaw', 'reflection', 'tbabs', 'gaussian', 'blackbody'], arf_filtered)
chi2 = calculate_chi_squared(rate_filtered, final_model_flux, error_filtered)
dof = len(rate_filtered) - len(params_5)
red_chi2 = chi2 / dof

print(f"\n📊 Final Statistics:")
print(f"  Chi-Squared     : {chi2:.2f}")
print(f"  Degrees of Freedom : {dof}")
print(f"  Reduced Chi-sq  : {red_chi2:.4f}")

# Evaluation
if 0.8 <= red_chi2 <= 1.2:
    print("\n🌟 Result: Excellent Fit! (โมเดลอธิบายข้อมูลได้ดีมาก)")
elif 1.2 < red_chi2 <= 2.0:
    print("\n✅ Result: Acceptable Fit (พอใช้ได้ อาจปรับจูนได้อีก)")
elif red_chi2 > 2.0:
    print("\n❌ Result: Bad Fit (โมเดลยังไม่ครอบคลุมข้อมูล หรือติด Noise)")
else:
    print("\n⚠️ Result: Overfitting (ค่า Chi^2 ต่ำเกินไป เช็ค Error bar)")